<a href="https://colab.research.google.com/github/kimwei/colabs/blob/main/hugging_face_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flan-T5

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece==0.1.94

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Wie ich er bitten?</s>


In [ ]:
input_text = "lowercase this English sentence: HOW OLD ARE YOU?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

<pad> WHICH ARE YOU?</s>


# ChatGLM2-6b-int4

In [ ]:
#@title Fix the error `NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968` issue.
#
# For details see https://github.com/googlecolab/colabtools/issues/3409
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!git clone https://github.com/THUDM/ChatGLM-6B
%cd ChatGLM-6B
!pip install -r requirements.txt

Cloning into 'ChatGLM-6B'...
remote: Enumerating objects: 1227, done.
remote: Counting objects: 100% (633/633), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 1227 (delta 563), reused 530 (delta 518), pack-reused 594
Receiving objects: 100% (1227/1227), 9.12 MiB | 13.69 MiB/s, done.
Resolving deltas: 100% (730/730), done.
/content/ChatGLM-6B
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 6.8 MB/s eta

In [ ]:
# For CPU model, check GCC version
!gcc --version

gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
!echo |cpp -fopenmp -dM |grep -i open

#define _OPENMP 201511


In [1]:
#@title GPU info
#
# If with FREE Google colab, the GPU is Nvidia Tesla T4 with 16GB GDDR6
!nvidia-smi

Sat Jul 22 17:02:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Fix the error `ImportError: cannot import name 'builder' from 'google.protobuf.internal'`
# during model inference, need to upgrade the protobuf version
# For details see https://stackoverflow.com/questions/71759248/importerror-cannot-import-name-builder-from-google-protobuf-internal
# !pip install --upgrade protobuf

In [ ]:
#@title verify GPU exists by pytorch
import torch

has_cuda = torch.cuda.is_available()
print("is_cuda_available ?", has_cuda)

if has_cuda:
  print("GPU device = ", torch.cuda.get_device_name(0))
else:
  print("No GPU available")

is_cuda_available ? False
No GPU available


In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True)
# GPU model
model = AutoModel.from_pretrained("THUDM/chatglm2-6b-int4", trust_remote_code=True).half().cuda()
# CPU model
# model = AutoModel.from_pretrained("THUDM/chatglm2-6b-int4",trust_remote_code=True).cpu().float()

model = model.eval()

In [ ]:
def predict(message:str) -> str:
  response, history = model.chat(tokenizer, message, history=[])
  return response

predict("现在是什么时间？")

'抱歉，作为一个人工智能助手，我没有实时的时间功能。'

In [ ]:
predict("你是什么时候训练的？")

'我是基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型 GLM2-6B 开发的，我的训练时间取决于我接受到的训练数据。'

In [ ]:
predict("白日依山尽的下一句是什么")

'白日依山尽,黄河入海流。这是唐代诗人王之涣的《登鹳雀楼》中的名句。'

In [ ]:
predict("白日依山尽的下一句的下一句是什么")

'白日依山尽,黄河入海流。这句话出自唐代诗人王之涣的《登鹳雀楼》。'

In [ ]:
predict("白日依山尽这首诗的题目是什么？")

'白日依山尽,黄河入海流。这是唐代著名诗人王之涣的《登鹳雀楼》。这首诗描述了诗人登上鹳雀楼时的所见所感,通过描绘山水的壮丽和宇宙的无限,表达了诗人对于人生的思考和对于自然的赞美。'

In [ ]:
predict("白日依山尽这首诗的作者是谁？")

'白日依山尽,黄河入海流这首诗的作者是唐代著名诗人王之涣。'

In [ ]:
predict("白日依山尽这首诗的题目以及作者分别是什么？")

'白日依山尽是唐代著名诗人王之涣所作，这首诗的题目为《登鹳雀楼》。这首诗通过描绘作者登高远眺的情景，抒发了作者对美好事物的向往和对人生苦难的感慨，被认为是唐代楼观诗的代表作之一。'

In [ ]:
predict("who is the previous president of the US prior to Biden? And who is his wife?")

"The previous president of the United States before Biden was President Donald Trump. He served two terms in office from 2017 to 2021. Trump's wife was Melania Trump."